In [48]:
from skimage.color import rgb2gray
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy import ndimage
import random as rng
from sklearn.cluster import KMeans
rng.seed(12345)
import random
import math

In [49]:
PATH_YES = "../images/YES"
PATH_NO = "../images/NO"

# Change PATH here
PATH = PATH_NO

files = []
for fname in os.listdir(PATH):
    if fname.endswith(".bmp"):
        files.append(fname)

files

['190825_141725_0001318852_CAM1_NG_NO.bmp',
 '190825_163433_0001726562_CAM1_NG_NO.bmp',
 '190825_141022_0001297871_CAM1_NG_NO.bmp',
 '190825_143755_0001379937_CAM1_NG_NO.bmp',
 '190825_145934_0001444421_CAM1_NG_NO.bmp',
 '190828_153935_0000950386_CAM2_NG_NO.bmp',
 '190829_110053_0004404780_CAM2_NG_NO.bmp',
 '190829_100100_0004227057_CAM2_NG_NO.bmp',
 '190825_191034_0002190225_CAM1_NG_NO.bmp',
 '190825_162806_0001707341_CAM1_NG_NO.bmp',
 '190828_132418_0000547436_CAM2_NG_NO.bmp',
 '190825_152819_0001529268_CAM1_NG_NO.bmp',
 '190825_214821_0002660156_CAM1_NG_NO.bmp',
 '190829_102118_0004286876_CAM2_NG_NO.bmp',
 '190829_134030_0004880169_CAM2_NG_NO.bmp',
 '190825_221741_0002747556_CAM1_NG_NO.bmp',
 '190828_103638_0000048722_CAM2_NG_NO.bmp',
 '190828_143048_0000745480_CAM2_NG_NO.bmp',
 '190825_203721_0002448710_CAM1_NG_NO.bmp',
 '190828_112841_0000203024_CAM2_NG_NO.bmp',
 '190825_145224_0001423067_CAM1_NG_NO.bmp',
 '190828_122749_0000379152_CAM2_NG_NO.bmp',
 '190825_165122_0001776326_CAM1_

In [50]:
#img = cv2.imread(PATH + '/' + files[9], 0)
#img_orig = cv2.imread(PATH + '/' + files[9], 1)
#plt.imshow(img, cmap='gray')

In [51]:
class Wire:   
            img = np.zeros([1200,1600], dtype="uint8")
            resized = np.zeros([600,800], dtype="uint8")
            x_locations = []
            top = -1
            bottom = -1
            def __init__(self, img):
                self.img = img  
                # choosing 10 x points randomly to feed the algorithm
                j = 0
                for i in range(img.shape[1]):
                    j = j+1
                    if j == 10:
                        break
                    r = random.SystemRandom()
                    number = r.randint(0,img.shape[1]) 
                    if number not in self.x_locations :
                        self.x_locations.append(i)
                self.resize_pic()        
                self.top = self.wire_frame(0) 
                self.bottom = self.wire_frame(1) 
        
            # A function to determine which y position is the most popular among final points y positions        
            def most_frequent(self,List): 
                counter = 0
                num = List[0] 
                for i in List: 
                    curr_frequency = List.count(i) 
                    if(curr_frequency> counter): 
                        counter = curr_frequency 
                        num = i 
                return num 
            

            def find_mean(self,x):
                mysum = 0
                points_y = []
                
                # we devide picture to 3 zones
                size = math.floor(self.resized.shape[0]/3)
                start = 0
                end = size
                for i in range(2):
                    for j in range(10):
                        r = random.SystemRandom()
                        number = r.randint(start,end) 
                        if number not in points_y:
                            points_y.append(number)
                    start = start + size
                    end = end + size
                
                for j in range(len(points_y)):
                    y = points_y[j]
                    pixel_value = self.resized[y,x]
                    mysum = mysum + pixel_value
                
                mean = np.mean(mysum/len(points_y))
                return mean
            
            def resize_pic(self):
                scale_percent = 50 # percent of original size
                width = int(self.img.shape[1] * scale_percent / 100)
                height = int(self.img.shape[0] * scale_percent / 100)
                dim = (width, height)
                resized = cv2.resize(self.img, dim, interpolation = cv2.INTER_AREA)
                self.resized = resized
                
            
            def wire_frame(self,arg):
                    y_values = []
                    # traversing the image with chosen x points, and checking pixel value against
                    #threshold = mean_value
                    # based on arg, start from 0:top or 1:bottom of the image 
                    if (arg == 0):
                         for i in self.x_locations:
                            mean_value = self.find_mean(i)
                            for j in range(self.resized.shape[0]):
                                pixel_value = self.resized[j,i]
                                if pixel_value > mean_value: 
                                    y_values.append(j)
                                    break;
                    
                    else:
                        for i in self.x_locations:
                            mean_value = self.find_mean(i)
                            for j in  reversed(range(self.resized.shape[0])):
                                pixel_value = self.resized[j,i]
                                if pixel_value > mean_value:
                                    y_values.append(j)
                                    break;

                    # final y position is the estimation of y by the algorithm
                    # * 2 because we scaled down the image before
                    found_y_position = self.most_frequent(y_values)
                    # * 2 because we scaled down the image before
                    found_y_position = found_y_position * 2
                    return found_y_position

In [52]:
mysum = 0
points_y = [1,2,3,4,5]
for j in range(len(points_y)):
    #print(j)
    mysum = mysum + points_y[j]
    
print(len(points_y))   
#import math
#math.floor(800/3)
#266+266+266

5


In [53]:
for i in range(len(files)-1):
    img = cv2.imread(PATH + '/' + files[i], 0)
    img_orig = cv2.imread(PATH + '/' + files[i], 1)
    wire_obj = Wire(img)
    top_position = wire_obj.top
    bottom_position = wire_obj.bottom
    lineThickness = 6
    cv2.line(img_orig, (0, top_position), (1599, top_position), (0,0,255), lineThickness)
    cv2.line(img_orig, (0, bottom_position), (1599, bottom_position), (0,0,255), lineThickness)
    #plt.imshow(img_orig)
    cv2.imwrite("result_"+str(i)+".bmp", img_orig) 

In [ ]:
points = [1,2,3,4,5]
for i in range(3):
    print(i)

In [ ]:
import numpy as np
PATH_YES = "../images/YES"
PATH_NO = "../images/NO"
# Change PATH here
PATH = PATH_YES
imag = cv2.imread(PATH + '/' + files[2], 0)

def find_min_max(x_location,img):
    min_value = 300
    max_value = 0
    min_point = ()
    max_point = ()
    for j in range(img.shape[0]):
        pixel_value = img[j,x_location]
        if pixel_value > max_value:
            max_value = pixel_value
            max_point = (j,i)
        if pixel_value < min_value:
            min_value = pixel_value
            min_point = (j,i)
    return min_value,max_value

x, y = find_min_max(0,imag)
print(x)
print(y)
avg = np.mean([x,y])
print(avg)

print(imag.shape[0])
print(imag.shape[1])

imag[1199,1599]

In [ ]:
import numpy as np
PATH_YES = "../images/YES"
PATH_NO = "../images/NO"
# Change PATH here
PATH = PATH_YES
imag = cv2.imread(PATH + '/' + files[2], 0)

def find_mean(x_location,img):
    mysum = 0
    
    for j in range(img.shape[0]):
        pixel_value = img[j,x_location]
        mysum = mysum + pixel_value
        
    
    mean = np.mean(mysum/img.shape[0])
    
    return mean

mean = find_mean(0,imag)
print(mean)


In [ ]:


def extractWire(img,avg):
    myimg = np.zeros([img.shape[0],img.shape[1]], dtype="uint8")
    for i in range(img.shape[1]-1):
        for j in range(img.shape[0]-1):
            if img[j,i] > avg:
                myimg[j,i] = img[j,i]
    return myimg
            

myimg = extractWire(img,mean)


for i in range(myimg.shape[0]):
    if myimg[i,200] > mean:
        print(i)
        break



#x, y = find_min_max(0,myimg)
plt.imshow(myimg)




#plt.imshow(myimg)
                
    #return i, k, img[i:k,:]
    #  if img[i,1] < 255:
    #        break


In [ ]:
  def find_mean_pixel_value(self,x_location):
                min_value = 300
                max_value = 0
               
                for j in range(self.img.shape[0]):
                    pixel_value = self.img[j,x_location]
                    if pixel_value < min_value:
                        min_value = pixel_value
                    if pixel_value > max_value:
                        max_value = pixel_value
                return np.mean(min_value,max_value)

In [ ]:
   def rezie_pic(self):
                scale_percent = 50 # percent of original size
                width = int(self.img.shape[1] * scale_percent / 100)
                height = int(self.img.shape[0] * scale_percent / 100)
                dim = (width, height)
                resized = cv2.resize(self.img, dim, interpolation = cv2.INTER_AREA)
                self.resized = resized
                return resized